# Advent of Code Challenges - Day 5

## Load Libraries

In [1]:
import numpy as np
import pandas as pd

from pandas import Series,DataFrame

## Load Data

In [2]:
# load data
input_path = './input.txt'

with open(input_path) as f:
    lines = f.readlines()
f.close()

# Clean and prepare data
data = []
for line in lines:
    points = line.replace("\n","").split("->")
    y1, x1 = points[0].strip().split(",")
    y2, x2 = points[1].strip().split(",")
    row = [x1,y1,x2,y2]
    data.append(row)

# Convert to a pandas DataFrame
df = DataFrame(data).astype(int)
df.columns = ['X1', 'y1', 'X2', 'y2']

# Calculate the maximum x and y values (num rows and num cols)
x_max = max([df.X1.max(), df.X2.max()])
y_max = max([df.y1.max(), df.y2.max()])

## Define Functions

In [3]:
def reset_map(x_max, y_max):
    """Create a dataframe that is 0 to x_max by 0 to y_max"""
    map_df = [[0]*(y_max+1) for x in range(0,x_max+1,1)]
    map_df = DataFrame(map_df)
    return(map_df)

In [4]:
def desc_line(line):
    """
    Function to return line starting at left side with slope
    i.e. returns the plot point with the lowest X value as the first pair of numbers along with 1 or -1
    """
    X1, y1, X2, y2 = line
    if X1 < X2:
        line_adj = [X1, y1, X2, y2]
        if y1 < y2:
            slope = 1
        else:
            slope = -1
    else:
        line_adj = [X2, y2, X1, y1]
        if y1 < y2:
            slope = -1
        else:
            slope = 1
    return(line_adj,slope)

## Puzzle 1 - Let's play Battleship!
1. Split the dataframe into a df with vertical lines an one with horizontal lines
2. Reset the map dataframe to all 0's.
3. Iterate through the horizontal and vertical lines and add 1 to each point on the map for each point on the line.
4. Sum the number of elements with value > 1

In [5]:
vert_df = df[df.y1 == df.y2]
vert_df.reset_index(drop=True, inplace=True)

horz_df = df[df.X1 == df.X2]
horz_df.reset_index(drop=True, inplace=True)

map_df = reset_map(x_max, y_max)
for line in horz_df.values:
    line = [[line[0],y] for y in range(min(line[1],line[3]),max(line[1],line[3])+1,1)]
    for point in line:
        map_df.iloc[point[0], point[1]] += 1

for line in vert_df.values:
    line = [[x,line[1]] for x in range(min(line[0],line[2]),max(line[0],line[2])+1,1)]
    for point in line:
         map_df.iloc[point[0], point[1]] += 1
            
# Answer is the count of elements in the map_df where the value is > 1
answer = (map_df > 1).sum(axis=1).sum()
print(f"Anser: {answer}")

Anser: 6113


## Puzzle 2 - You can't put your ship on a diagonal!
Reset data so puzzle 2 can be run from **here** down.

In [6]:
vert_df = df[df.y1 == df.y2]
vert_df.reset_index(drop=True, inplace=True)

horz_df = df[df.X1 == df.X2]
horz_df.reset_index(drop=True, inplace=True)

diag_df = df[df.X1 != df.X2]
diag_df.reset_index(drop=True, inplace=True)
diag_df = diag_df[diag_df.y1 != diag_df.y2]
diag_df.reset_index(drop=True, inplace=True)

In [7]:
# verify that diagonal line count matches with original shape less horizontal and vertial line counts
print(f"Vertical Lines: {vert_df.shape[0]}")
print(f"Horizontal Lines: {horz_df.shape[0]}")
print(f"Diagonal Lines: {diag_df.shape[0]}")
print(f"Count All Lines: {vert_df.shape[0] + horz_df.shape[0] + diag_df.shape[0]}")

Vertical Lines: 152
Horizontal Lines: 177
Diagonal Lines: 171
Count All Lines: 500


1. Split the dataframe into a df with vertical lines an one with horizontal lines
2. Reset the map dataframe to all 0's.
3. Iterate through the horizontal, vertical, and diagonal lines and add 1 to each point on the map for each point on the line.
4. Sum the number of elements with value > 1

In [8]:
map_df = reset_map(x_max, y_max)
for line in horz_df.values:
    line = [[line[0],y] for y in range(min(line[1],line[3]),max(line[1],line[3])+1,1)]
    for point in line:
        map_df.iloc[point[0], point[1]] += 1

for line in vert_df.values:
    line = [[x,line[1]] for x in range(min(line[0],line[2]),max(line[0],line[2])+1,1)]
    for point in line:
         map_df.iloc[point[0], point[1]] += 1

for line in diag_df.values:
    line_adj, slope = desc_line(line)
    x0, y0 = line_adj[0:2]
    line_len = abs(line_adj[0]-line_adj[2])
    if slope == 1:
        line_pts = [[x0+n, y0+n] for n in range(0,line_len+1,1)]
    else:
        line_pts = [[x0+n, y0-n] for n in range(0,line_len+1,1)]
    for point in line_pts:
        map_df.iloc[point[0], point[1]] += 1
        
# Answer is the count of elements in the map_df where the value is > 1
answer = (map_df > 1).sum(axis=1).sum()
print(f"Answer: {answer}")

Answer: 20373
